In [5]:
!pip install pytest-mock

In [6]:
!pip install pytest pytest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.3/342.3 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully uninstalled pytest-7.4.4


In [7]:
%%writefile stop_limit_order.py
import asyncio
from datetime import datetime

# Mock classes

class OrderType:
    StopLimit = 'StopLimit'

class Security:
    def __init__(self, symbol, price):
        self.symbol = symbol
        self.price = price

class Order:
    def __init__(self, symbol, quantity, time, tag='', properties=None):
        self.symbol = symbol
        self.quantity = quantity
        self.time = time
        self.tag = tag
        self.properties = properties

# Main StopLimitOrder class

class StopLimitOrder(Order):
    def __init__(self, symbol, quantity, stop_price, limit_price, time, tag='', properties=None):
        super().__init__(symbol, quantity, time, tag, properties)
        self.stop_price = stop_price
        self.limit_price = limit_price
        self.stop_triggered = False

    @property
    def type(self):
        return OrderType.StopLimit

    async def get_value(self, security):

        if self.quantity < 0:

            return self.quantity * max(self.limit_price, security.price)
        elif self.quantity > 0:

            return self.quantity * min(self.limit_price, security.price)
        return 0

    async def apply_update(self, update_request):

        if 'stop_price' in update_request:
            self.stop_price = update_request['stop_price']
        if 'limit_price' in update_request:
            self.limit_price = update_request['limit_price']

    def clone(self):
        return StopLimitOrder(
            self.symbol, self.quantity, self.stop_price, self.limit_price, self.time, self.tag, self.properties
        )




Overwriting stop_limit_order.py


In [10]:
%%writefile test_stop_limit_order.py
import pytest
from unittest.mock import AsyncMock
from stop_limit_order import StopLimitOrder, Security

@pytest.mark.asyncio
async def test_get_value_buy(mocker):
    security = Security(symbol="AAPL", price=140.0)
    order = StopLimitOrder(symbol="AAPL", quantity=10, stop_price=150.0, limit_price=155.0, time=None)

    mocker.patch.object(security, 'price', 140.0)

    value = await order.get_value(security)

    assert value == 1400.0

@pytest.mark.asyncio
async def test_get_value_sell(mocker):
    security = Security(symbol="AAPL", price=160.0)
    order = StopLimitOrder(symbol="AAPL", quantity=-5, stop_price=150.0, limit_price=155.0, time=None)

    
    mocker.patch.object(security, 'price', 160.0)

    value = await order.get_value(security)

    assert value == -800.0

@pytest.mark.asyncio
async def test_apply_update():
    order = StopLimitOrder(symbol="AAPL", quantity=10, stop_price=150.0, limit_price=155.0, time=None)

    update_request = {
        'stop_price': 145.0,
        'limit_price': 152.0
    }

    await order.apply_update(update_request)

   
    assert order.stop_price == 145.0
    assert order.limit_price == 152.0


@pytest.mark.asyncio
async def test_clone(mocker):
    order = StopLimitOrder(symbol="AAPL", quantity=10, stop_price=150.0, limit_price=155.0, time=None)

    cloned_order = order.clone()

    assert cloned_order.symbol == order.symbol
    assert cloned_order.quantity == order.quantity
    assert cloned_order.stop_price == order.stop_price
    assert cloned_order.limit_price == order.limit_price
    assert cloned_order.tag == order.tag
    assert cloned_order.properties == order.properties





Overwriting test_stop_limit_order.py


In [11]:
!pytest test_stop_limit_order.py --asyncio-mode=auto

/usr/local/lib/python3.10/dist-packages/pytest_asyncio/plugin.py:208: PytestDeprecationWarning: The configuration option "asyncio_default_fixture_loop_scope" is unset.
The event loop scope for asynchronous fixtures will default to the fixture caching scope. Future versions of pytest-asyncio will default the loop scope for asynchronous fixtures to function scope. Set the default fixture loop scope explicitly in order to avoid unexpected behavior in the future. Valid fixture loop scopes are: "function", "class", "module", "package", "session"

  warnings.warn(PytestDeprecationWarning(_DEFAULT_FIXTURE_LOOP_SCOPE_UNSET))
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0
rootdir: /content
plugins: asyncio-0.24.0, mock-3.14.0, anyio-3.7.1, typeguard-4.3.0
asyncio: mode=auto, default_loop_scope=None
collected 4 items                                                                   